The project was to determine which Pittsburgh neighborhood would be best for children, and so I choose to see which neighborhoods had the best schools. I determined this by finding the percentage of kids that were enrolled in the school's gifted programs. 
I am using the dataset:
Schools in pittsburgh cities: IEP by neighborhood
https://data.wprdc.org/dataset/pittsburgh-public-schools-individualized-education-plan-iep/resource/62403582-e6c3-40b6-9bb6-d5d38e3f058d

In [1]:
import pandas as pd
import geopandas
schools = pd.read_csv("neighborhood_iep.csv")
schools['percentage_of_gifted'] = round((schools['gifted'] / schools['total_enrollment_k_12']) * 100, 2)
df = pd.DataFrame(schools).sort_values(by=['percentage_of_gifted'], ascending=False).set_index('neighborhoods')
##dictionary = {'Neighborhoods':[], 'Percentage_of_Gifted':[]}
neighborhood_list = []
gifted_list = []

for index, row in schools.iterrows():
    neighborhoods = row['neighborhoods']
    gifted = row['percentage_of_gifted']
    split = neighborhoods.split(", ")
    length = len(split)
    i = 1
    while i < len(split):
        neighborhood_list.append(split[i])
        gifted_list.append(gifted)
        i += 1
dictionary = {'Neighborhoods':neighborhood_list, 'Percentage_of_Gifted':gifted_list}
df2 = pd.DataFrame(dictionary, columns=['Neighborhoods', 'Percentage_of_Gifted']).set_index('Neighborhoods').sort_values(by=['Percentage_of_Gifted'], ascending=False)
        
df2

,Percentage_of_Gifted
Neighborhoods,
Swisshelm Park,32.22
Regent Square,32.22
Friendship,14.78
Upper Lawrenceville,13.62
Strip District,8.15
Polish Hill,8.15
New Homestead,7.87
Lincoln Place,7.87
Troy Hill,6.62


In [4]:
neighborhoods = geopandas.read_file('Neighborhoods_.shp')
neighborhoods_map = neighborhoods.merge(schools, how='left', left_on='hood', right_on='neighborhoods')
neighborhoods_map.head()

KeyError: 'hood'

Seeing the percentage of the gifted students enrolled in each neighborhood, we can see that Squirrel Hill North has the best schools out of all the Pittsburgh neighborhoods, and is the best neighborhood to raise children in according to the data.